In [1]:
import numpy as np
import data

In [2]:
train = data.train
test = data.test

In [7]:
class MatrixFactorization():
    
    def __init__(self, train, test, f, epsilon):
        """
        param train : Rating Matrix for train
        param test : Rating Matrix for test
        param f : latent feature parameter
        """
        
        self._R = train # Implication Matrix for training size (m, n)
        self._R_test = test # Implication Matrix for test size (m, n)
        self._P = np.array(np.vectorize(lambda x: 0 if x==0 else 1)(train), dtype = np.float64) # Preference Matrix for training
        self._P_test = np.array(np.vectorize(lambda x: 0 if x==0 else 1)(test), dtype = np.float64) # Preference Matrix for training
        self._n_user_rated = np.sum(self._P, axis = 1)
        self._n_item_rated = np.sum(self._P, axis = 0)
        self._num_users, self._num_items = train.shape
        self._alpha = 0.01
        self._lambda = 0.01
        self._C = 1 + self._alpha * self._R # Confidence Matrix size (m, n)
        self._f = f
        self._epsilon = epsilon
        
        
    def fit(self):
        """
        training Matrix Factorization : update matrix latent weight and bias
        """
        # init latent features
        self._X = np.random.normal(0, 0.1, size=(self._num_users, self._f))
        self._Y = np.random.normal(0, 0.1, size=(self._num_items, self._f))
        
        cost_diff = 100000
        count = 0
        self._training_process = [100000000]
        # repeat ALS until convergence
        while cost_diff > self._epsilon :
        
            count += 1
            
            self._yTy = self._Y.T.dot(self._Y)
            for u in range(self._num_users):
                self.optimize_x(u)
            
            self._xTx = self._X.T.dot(self._X)
            for i in range(self._num_items):
                self.optimize_y(i)
    
            cost = self.cost()
            self._training_process.append(cost)
            cost_diff = self._training_process[count - 1] - self._training_process[count]
            rank = self.compute_rank()
            print("count: %d, cost_difference : %.4f, rank : %.4f"% (count, cost_diff, rank))
        
        self.print_results()
                
                
    def optimize_x(self, u):
        """
        Optimize X given user u
        """
        C_u = np.diag(self._C[u, :]) # create diagonal matrix size (n, n)
        
        # (f,f) matrix
        temp1 = self._yTy + self._Y.T.dot(C_u - np.identity(self._num_items)).dot(self._Y) + self._lambda * np.identity(self._f)
        # (f,1) matrix
        temp2 = self._Y.T.dot(C_u).dot(self._P[u])
        
        self._X[u, :] = np.linalg.inv(temp1).dot(temp2)
        
    
    def optimize_y(self, i):
        """
        Optimize X given user u
        """
        C_i = np.diag(self._C[:, i]) # create diagonal matrix size (m, m)
        
        # (f,f) matrix
        temp1 = self._xTx + self._X.T.dot(C_i - np.identity(self._num_users)).dot(self._X) + self._lambda * np.identity(self._f)
        # (f,1) matrix
        temp2 = self._X.T.dot(C_i).dot(self._P[:, i])
        
        self._Y[i, :] = np.linalg.inv(temp1).dot(temp2)
        
                
    def cost(self):
        """
        compute Loss function
        """
        loss = np.sum(self._C * np.square(self._P - self._X.dot(self._Y.T))) + self._lambda * (np.linalg.norm(self._X) + np.linalg.norm(self._Y))
        
        return loss
    
    
    def compute_rank(self):
        
        prediction = self._X.dot(self._Y.T)
        sort_mat = prediction.argsort()
        rank_mat = sort_mat / self._num_items
        
        test_x, test_y = self._R_test.nonzero()
        temp_1 = 0
        temp_2 = 0
        for x, y in zip(test_x, test_y):
            temp_1 += self._R_test[x, y] * rank_mat[x, y] 
            temp_2 += self._R_test[x, y]
        
        rank = temp_1 / temp_2
            
        return rank
    
    
    def print_results(self):
        """
        print fit results
        """

        print("Final P hat matrix:")
        print(self._X.dot(self._Y.T))

In [8]:
np.random.seed(7)
    
np.seterr(all="warn")
    
factorizer = MatrixFactorization(train, test, f=40, epsilon = 1.0)
factorizer.fit()

count: 1, cost_difference : 99949380.6194, rank : 0.5630
count: 2, cost_difference : 8738.1106, rank : 0.5544
count: 3, cost_difference : 1341.1171, rank : 0.5524
count: 4, cost_difference : 473.3875, rank : 0.5500
count: 5, cost_difference : 221.1716, rank : 0.5508
count: 6, cost_difference : 120.6559, rank : 0.5488
count: 7, cost_difference : 73.4669, rank : 0.5526
count: 8, cost_difference : 48.5623, rank : 0.5511
count: 9, cost_difference : 34.1283, rank : 0.5489
count: 10, cost_difference : 25.1087, rank : 0.5495
count: 11, cost_difference : 19.1165, rank : 0.5521
count: 12, cost_difference : 14.9315, rank : 0.5502
count: 13, cost_difference : 11.8894, rank : 0.5476
count: 14, cost_difference : 9.6087, rank : 0.5483
count: 15, cost_difference : 7.8585, rank : 0.5508
count: 16, cost_difference : 6.4918, rank : 0.5504
count: 17, cost_difference : 5.4105, rank : 0.5470
count: 18, cost_difference : 4.5459, rank : 0.5465
count: 19, cost_difference : 3.8485, rank : 0.5514
count: 20, cos